In [1]:
import cv2
import numpy as np

In [73]:
def model_recognition(img, template):
    # Convert both images to grayscale
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

    # Initialize the SIFT feature detector and descriptor extractor
    sift = cv2.xfeatures2d.SIFT_create()

    # Detect keypoints and extract descriptors from both images
    kp1, des1 = sift.detectAndCompute(template_gray, None)
    kp2, des2 = sift.detectAndCompute(img_gray, None)

    # Initialize the Brute-Force Matcher and match the descriptors between the two images
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
    matches = bf.match(des1, des2)

    # Select only the good matches based on a threshold
    good_matches = []
    for m in matches:
        if m.distance < 0.5 * min(len(des1), len(des2)):
            good_matches.append(m)

    # Calculate the transformation between the object template and the image using the good matches
    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)
    if src_pts.size>6 and dst_pts.size>6:
        M, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        if M.size>0:

            # Draw a bounding box around the object in the image using the calculated transformation
            h, w = template_gray.shape
            pts = np.float32([ [0,0], [0,h-1], [w-1,h-1], [w-1,0] ]).reshape(-1,1,2)
            dst = cv2.perspectiveTransform(pts, M)
            img = cv2.polylines(img, [np.int32(dst)], True, (0,0,255), 2)
            return img
        else:
            return np.array([])
    else:
        return np.array([])


In [75]:
import os
templates = os.listdir('sprites/alchemist')
t = cv2.imread('sprites/Alchemist_Sprite.png')
img = cv2.imread('screens/ikran3.png')[:870, :]
for x in templates:
    tmplt = cv2.imread(f'sprites/alchemist/{x}')
    tmplt = cv2.resize(tmplt, (120,140), interpolation=cv2.INTER_NEAREST)
    res = model_recognition(img, tmplt)
    if res.any() != 0:
        cv2.imshow('result', res)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        break